In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/data1.csv',names=['movie', 'user','rating','date'],nrows=100000)

In [ ]:
data.shape

(100000, 4)

In [ ]:
data.head()

,movie,user,rating,date
0,1,1488844,3,2005-09-06
1,1,822109,5,2005-05-13
2,1,885013,4,2005-10-19
3,1,30878,4,2005-12-26
4,1,823519,3,2004-05-03


In [ ]:
import pandas as pd
data.sort_values(by=['user','date'],ascending=True,inplace=True)

In [ ]:
data['last_movie_rating'] = data.groupby(['user'],sort=False)['rating'].shift()

In [ ]:
data['last_movie_watched'] = data.groupby(['user'],sort=False)['movie'].shift()

In [ ]:
data.fillna(0,inplace=True)

In [ ]:

data.head()

,movie,user,rating,date,last_movie_rating,last_movie_watched
72295,28,7,4,2005-05-23,0.0,0.0
12541,8,7,5,2005-07-30,4.0,28.0
77793,28,134,5,2004-11-09,0.0,0.0
69061,28,201,4,2003-01-23,0.0,0.0
93726,30,261,4,2004-09-21,0.0,0.0


In [ ]:
y= data['rating'].values


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
model= OneHotEncoder(handle_unknown="ignore")
#model= OneHotEncoder()
model1= OneHotEncoder()
x_train = model.fit_transform(data[0:80000][['user',"movie"]])

In [ ]:
x_train.shape

(80000, 65283)

In [ ]:
x_test = model.transform(data[80000:][['user',"movie"]])

In [ ]:
x_test.shape

(20000, 65283)

In [ ]:
#from fastFM2.datasets import make_user_item_regression
import scipy.sparse as sp
from fastfm2 import als
import fastfm2
n_iter = 10000
rank = 50
seed = 333
step_size = 1
l2_reg_w = 0.001
l2_reg_V = 0.001

fm = fastfm2.als.FMRegression(n_iter=1000, l2_reg_w=l2_reg_w,
            l2_reg_V=l2_reg_V, rank=rank, random_state=seed)
    # initalize coefs
fm.fit(sp.csc_matrix(x_train),y[0:80000])
#(sp.csc_matrix(x_train),y)

FMRegression(init_stdev=0.1, l2_reg=0, l2_reg_V=0.001, l2_reg_w=0.001,
             n_iter=1000, random_state=333, rank=50)

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np
pred = fm.predict(sp.csc_matrix(x_train))
print("train mse",mean_squared_error(pred, y[0:80000]))
print("train rmse",np.sqrt(mean_squared_error(pred, y[0:80000])))

train mse 3.363904185297393e-10
train rmse 1.8340949226518765e-05


In [ ]:
pred = fm.predict(sp.csc_matrix(x_test))
print("train mse",mean_squared_error(pred, y[80000:]))
print("train rmse",np.sqrt(mean_squared_error(pred, y[80000:])))

train mse 1.1302572932831536
train rmse 1.063135594965738
